In [ ]:
# General import
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from PIL import Image
from scipy import stats
import datetime
from timeit import default_timer as timer
from sklearn.utils import shuffle

# Keras 1.2.2 import
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, ActivityRegularization
from keras.layers import Convolution2D, MaxPooling2D, Convolution3D, MaxPooling3D
from keras.utils import np_utils
from keras.optimizers import SGD, Adagrad, Adam
from keras.regularizers import l1
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import keras.backend as K

In [ ]:
# Functions Definition:

# Read data from csv file function
def read_data(file_path):
    #column_names = ['user','timestamp', 'x-axis', 'y-axis', 'z-axis', 'label']
    data = pd.read_csv(file_path, header = 0, sep = ',')#, names = column_names)
    return data

def tail(f, n):
    assert n >= 0
    pos, lines = n+1, []
    while len(lines) <= n:
        try:
            f.seek(-pos, 2)
        except IOError:
            f.seek(0)
            break
        finally:
            lines = list(f)
        pos *= 2
    return lines[-n:]

def time_el(pre):
    
    now = datetime.datetime.now() # Today complete date and time
    
    return ((now.day - pre[0])*24*60) + ((now.hour - pre[1])*60) + (now.minute - pre[2])

def size_keras(model): # Compute number of params in a model (the actual number of floats)
    return sum([np.prod(K.get_value(w).shape) for w in model.trainable_weights])

def save_result_line(filename, results):

    # Getting last used ID number
    f = open(filename)
    last_line = tail(f, 1)
    last_results = np.genfromtxt(last_line, delimiter='\t')
    f.close
    
    if last_line == []:
        results[0] = 1
    else:
        results[0] = last_results[0] + 1
    
    # Saving current result line
    f = open(filename, 'ab') 
    np.savetxt(f, np.reshape(results, (1,-1)), delimiter='\t')
    f.close()
    
    return results[0]

def loss_rank_one(yt, yp, margin_value):
    
    s = yt * 1
    m1 = np.argmax(s)
    s[m1] = 0
    m2 = np.argmax(s)
    
    max1 = np.ones(len(yt)) * yp[m1]
    max2 = np.ones(len(yt)) * yp[m2]
    margin = np.ones(len(yt)) * margin_value
    
    loss = 2 * np.sum(np.log(1 + np.exp(-(max1 - yp - margin))))
    loss += np.sum(np.log(1 + np.exp(-(max2 - yp - margin))))
    loss += - np.sum(np.log(1 + np.exp(-(yp[m2] - yp[m1] - margin_value))))
    
    return loss
                
def loss_rank(yt_list, yp_list, margin_value):
    
    m = yp_list.shape[0]
    C_loss = 0
    
    for jj in range(0, m):
        C_loss = C_loss + loss_rank_one(yt_list[jj, :], yp_list[jj, :], margin_value)
        
    C_loss = (C_loss / m)

    return C_loss

# For multiple samples (numpy 2D array input)
def loss_keras_margin(margin_value):
    def loss_keras(yt, yp):

        # 1st most dominant - Guessing 1st dominant is more important
        ind_max = K.argmax(yt, 1)
        flat_y = K.reshape(yp, [-1])  # Reshape to a vector.

        flat_ind_max = ind_max + K.cast(tf.range(tf.shape(yp)[0]) * tf.shape(yp)[1], tf.int64)
        y_m1 = K.reshape(K.tf.gather(flat_y, flat_ind_max), [-1, 1])
        
        y_mm1 = K.repeat_elements(y_m1, K.int_shape(yp)[1], 1)

        # 2nd most dominant
        ind_min = K.argmin(yt, 1)
        flat_y = K.reshape(yp, [-1])  # Reshape to a vector.

        flat_ind_min = ind_min + K.cast(tf.range(tf.shape(yp)[0]) * tf.shape(yp)[1], tf.int64)
        y_m2 = K.reshape(K.tf.gather(flat_y, flat_ind_max), [-1, 1])
        y_mm2 = K.repeat_elements(y_m2, K.int_shape(yp)[1], 1)

        
        margin = (y_mm2/y_mm2) * margin_value

        loss = (2 * K.sum(K.mean(K.log(1 + K.exp(-(y_mm1 - yp - margin))), 0)) \
                + K.sum(K.mean(K.log(1 + K.exp(-(y_mm2 - yp - margin))),0)) \
                - K.sum(K.mean(K.log(1 + K.exp(-(y_m2 - y_m1 - margin[:,0]))),0)))

        return loss

    return loss_keras # Returns loss function it self

def dcg_at_k(r, k, method=0):

    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def eval_custom(yt, ypp, margin_value):
    
    # If it is baseline, copy one vector to all spots, making yt same size as yp
    if len(ypp) == len(yt) :
        yp = ypp
    else :
        yp = []
        for i_h in range(0,len(yt)):
            yp.append(ypp)
        yp = np.array(yp)
    
    # KLD metric
    kld = 0.0
    for ii in range(0, len(yt)):
        #temp = yt[ii, :] != 0 # This eliminates points where yt is predicted to be 0
        #kld = kld + (np.sum(stats.entropy(yt[ii, temp], yp[ii, temp])) / len(yt))
        kld = kld + (np.sum(stats.entropy(yt[ii, :], yp[ii, :])) / len(yt))
    
    # 1st dominant accuracy
    dom1 = np.sum((np.argmax(yt,1) == np.argmax(yp,1))*1.0) /  len(yt)
    
    # 1st+2nd dominant accuracy
    st = yt * 1
    sp = yp * 1
    for ii in range(0, len(yt)):
        st[ii][ np.argmax(st[ii,:]) ] = 0
        sp[ii][ np.argmax(sp[ii,:]) ] = 0
    dom12 = np.sum(((np.argmax(st,1) == np.argmax(sp,1)) & (np.argmax(yt,1) == np.argmax(yp,1)))*1.0) /  len(yt)
    
    # Discounted Cumulative Gain
    dcg = 0
    for ii in range(0, len(yt)):
        s = np.argsort(yt[ii,:])
        sp = np.zeros(yt.shape[1])
        for jj in range(0, yt.shape[1]):
            sp[jj] = yp[ii, s[jj]]
        dcg = dcg + (dcg_at_k(sp, yt.shape[1]) / len(yt))
    
    # Logaritmic rank loss
    rank = loss_rank(yt, yp, margin_value)
    
    return [kld, dom1, dom12, dcg, rank]

In [ ]:
# Keras model definition function
def new_model(x_dim, classes, reg, l_rate, k_mod, obj, margin_value):
    
    model = Sequential()
    
    if k_mod == 0 : # -> 'linear'
        model.add(Flatten(input_shape=(x_dim[0],x_dim[1],1)))
        model.add(Dense(classes, activation = 'softmax', W_regularizer = l1(reg), init='he_normal'))
    
    if k_mod == 1 : # -> 'dnn_1'
        model.add(Flatten(input_shape=(x_dim[0],x_dim[1],1)))
        model.add(Dense(50, activation = 'relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation = 'softmax', W_regularizer = l1(reg), init='he_normal'))
    
    if k_mod == 2 : # -> 'dnn_2'
        model.add(Flatten(input_shape=(x_dim[0],x_dim[1],1)))
        model.add(Dense(1000, activation = 'relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(50, activation = 'relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation = 'softmax', W_regularizer = l1(reg), init='he_normal'))
    
    if k_mod == 3 : # -> 'dnn_3'
        model.add(Flatten(input_shape=(x_dim[0],x_dim[1],1)))
        model.add(Dense(100, activation = 'relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(100, activation = 'relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(50, activation = 'relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation = 'softmax', W_regularizer = l1(reg), init='he_normal'))
    
    if k_mod == 5 : # -> 'cnn_1'
        model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', 
                                input_shape=(x_dim[0],x_dim[1],1)))
        model.add(Convolution2D(16, 3, 3, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(50, activation='relu', W_regularizer = l1(reg), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation='softmax', W_regularizer = l1(reg), init='he_normal'))

    if k_mod == 6 : # -> 'cnn_2'
        model.add(Convolution2D(32, 3, 3, activation='relu',  border_mode='same',
                                input_shape=(x_dim[0], x_dim[1], 1)))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(16,3 ,3 , activation='relu', border_mode='same', ))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(16, 3, 3, activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(50, activation='relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation='softmax', W_regularizer = l1(reg), init='he_normal'))
    
    if k_mod == 7 : # -> 'cnn_3D'
        # Only for raw input x data, it needs to be reshaped
        model.add(Reshape((x_dim[2], x_dim[3], x_dim[4], 1), input_shape=(x_dim[0], x_dim[1], 1)))
        model.add(Convolution3D(32, 3, 3, 3, activation='relu',  border_mode='same',))
        model.add(MaxPooling3D(pool_size=(2,2,2)))
        model.add(Convolution3D(16, 3 ,3, 3, activation='relu', border_mode='same', ))
        model.add(MaxPooling3D(pool_size=(2,2,2)))
        model.add(Convolution3D(8, 3, 3, 3, activation='relu'))
        model.add(MaxPooling3D(pool_size=(2,2,2)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(50, activation='relu', W_regularizer = l1(0.001), init='he_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation='softmax', W_regularizer = l1(reg), init='he_normal'))
    
    # Compilation of model
    if obj < 2:
        model.compile(loss = 'kullback_leibler_divergence',
                  optimizer = Adam(lr=l_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                  metrics = ['kullback_leibler_divergence'])
    
    if obj == 2:
        model.compile(loss = 'categorical_crossentropy',
                  optimizer = Adam(lr=l_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                  metrics = ['accuracy'])
        
    if obj == 3:
        model.compile(loss = loss_keras_margin(margin_value), \
                  optimizer = Adam(lr=l_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), \
                  metrics = ['kullback_leibler_divergence'])
    
    return model

In [ ]:
def one_run(n_epoch, av_epoch, random_set, cut, nnz_01, l_rate, reg_lambda, k_mod, obj, inp, m_v, prin_t):
    
    # Starting timer, to measure how long it takes to generate this one run
    tim_start = timer()

    # Output variable declaration
    metadata = 20
    n_metric = 5 # (KLD, 1st dom, 1+2nd dom, DCG, Log loss 1, Log l. 2, Log l. 3, Log l. 4, Log l. 5, reserve)
    baseline = 4 * n_metric
    results = np.zeros(metadata + baseline + int(((n_epoch/av_epoch)*2*n_metric)))
    
    ## Saving Meta-Data to current Results line
    # Time information
    now = datetime.datetime.now() # Today complete date and time
    results[1] = now.year
    results[2] = now.month
    results[3] = now.day
    results[4] = now.hour
    results[5] = now.minute
    #results[6] will be finally filled with elapsed time in minutes
    #results[7] will be finally filled with elapsed time from timer
    #results[8] will be finally filled with total number of parameters in used keras model
    
    # If the objective is not our custom rank loss function, set margin to 0
    if obj != 3:
        m_v = 0
    
    # Given settings
    results[9:20] = [n_epoch, av_epoch, random_set, cut, (nnz_01*1), l_rate, reg_lambda, k_mod, obj, inp, m_v]
    
    ## ------------------------------------------------------------------------------------------------
    
    # Load data from files
    if (inp == 0) | (inp == 2): # Means Histogram input is selected
        # Borders to cut Histograms
        border_cut = read_data('IN_borders.csv')
        border = np.array(border_cut[0:])  # Casting as numpy array
        # Input Histograms
        inp_hist = read_data('IN_histogram.csv')
        data_inp = np.array(inp_hist[0:]) * 1.0 # Making sure that our array is float type (no integer)
        data_inp = data_inp[0:-1] # One data point is discarded to make processing easier (110/5 = 22)
        # Output labels
        out_labels = read_data('IN_labels.csv') # Output true labels
        data_out_raw = np.array(out_labels)  # Casting as numpy array
        data_out_raw = data_out_raw[0:-1] # One data point is discarded to make processing easier (110/5 = 22)
    else:
        # Input Raw 3D data
        f = open('IN_labels_raw_input.dat')
        data_out_raw = np.genfromtxt(f, delimiter='\t')
        f.close
        # Input Raw 3D data
        f = open('IN_raw_input.dat')
        data_inp = np.genfromtxt(f, delimiter='\t')
        f.close
        data_inp = np.reshape(data_inp, (-1, data_inp.shape[1], 1, 1))
        cut = -1 # To prevent cutting the input data
        
    # Reducing labels to 3 classes (when: inp = 2, inp = 3)
    if inp > 1:
        temp = data_out_raw * 1.0
        data_out_raw = np.zeros([temp.shape[0], 3])
        data_out_raw[:,0] = temp[:,0]
        data_out_raw[:,1] = temp[:,1] + temp[:,2]
        data_out_raw[:,2] = temp[:,3] + temp[:,4]
        # Renormalize to 1
        for ii in range(0, data_out_raw.shape[0]):
            data_out_raw[ii, :] = data_out_raw[ii, :] / np.sum(data_out_raw[ii, :])
    
    ## Processing labels for different objectives
    # No change to output labels
    if obj == 0:
        data_out = data_out_raw * 1
    
    # Keep just 1st most dominant type with value 1 (1 for 1st dominant, zero elsewhere)
    if obj == 1:
        temp = np.argmax(data_out_raw, axis=-1)
        data_out = np_utils.to_categorical(temp, data_out_raw.shape[1])
        
    # Keep just 1st and 2nd dominant type, renormalize to 1 ([0|0.2|0.1|0.15|0.5|0.05]->[0|0.2|0|0|0.5|0]/0.7)
    if obj == 2:
        temp = data_out_raw * 1.0
        max1 = np.argmax(temp, 1)
        for jj in range(0, temp.shape[0]):
            temp[jj, max1[jj]] = 0
        max2 = np.argmax(temp, 1)
        data_out = data_out_raw * 0.0
        for jj in range(0, data_out.shape[0]):
            data_out[jj, max1[jj]] = data_out_raw[jj, max1[jj]]
            data_out[jj, max2[jj]] = data_out_raw[jj, max2[jj]]
            data_out[jj, :] = data_out[jj, :] / np.sum(data_out[jj, :])
        
    # Create special ranking vector for my custom loss function (1st = 1, 2nd = -1, else = 0)
    if obj == 3:
        temp = data_out_raw * 1.0
        max1 = np.argmax(temp, 1)
        for jj in range(0, temp.shape[0]):
            temp[jj, max1[jj]] = 0
        max2 = np.argmax(temp, 1)
        data_out = temp * 0.0
        for jj in range(0, temp.shape[0]):
            data_out[jj, max1[jj]] = 1
        for jj in range(0, temp.shape[0]):
            data_out[jj, max2[jj]] = -1
    
    ## Processing input
    # Choose the compression (Cutting the input)
    if (cut <= 100) & (cut > 0) : # If Raw input is used, cut variable is set to -1
        condition = border >= cut
        correct_borders = border[condition[:,-1], :]
        bcut = correct_borders[-1,0:4]
        data_inp = np.reshape(data_inp, (-1, int(border[0,1]), int(border[0,3]), 1))
        data_inp = data_inp[:, bcut[0]:bcut[1], bcut[2]:bcut[3], :]

    # Useful variables
    x_dim = np.zeros(5, dtype=np.int32)
    x_dim[0:2] = [data_inp.shape[1], data_inp.shape[2]]
    if (inp == 1) | (inp == 3): # Means Raw input is selected
        x_dim[2:5] = [70, 83, 80] # Size of raw input (70x83x40 for iodine and vnc)
    samples = data_out.shape[0]
    classes = data_out.shape[1]
    set_size = (samples / 5) # There are 5 CV sets
        
    # Replacing all Non-Zero elements with 1
    if nnz_01 == True :
        data_inp = np.array(data_inp, dtype=bool) * 1.0
    
    # Shuffle of data segments
    data_inp, data_out = shuffle(data_inp, data_out, random_state = int(random_set)) # Pseudo-random shaffling
    
    # Dataset split to 5 mini sets
    X_split = np.zeros((5, set_size, x_dim[0], x_dim[1], 1))
    y_split = np.zeros((5, set_size, classes))
    y_split_raw = np.zeros((5, set_size, classes))
    for i_split in range(0, 5):
        X_split[i_split, :, :, :] = data_inp[i_split*set_size:set_size+i_split*set_size, :, :, :]
        y_split[i_split, :, :] = data_out[i_split*set_size:set_size+i_split*set_size, :]
        y_split_raw[i_split, :, :] = data_out_raw[i_split*set_size:set_size+i_split*set_size, :]

    # Variables for final model training
    Xa = np.zeros((set_size*4, x_dim[0], x_dim[1], 1)) # Xa is moved to X during normalization,
    X = np.zeros((set_size*4, x_dim[0], x_dim[1], 1))  # then Xa is not used
    y = np.zeros((set_size*4, classes))
     
    Xa_test = np.zeros((set_size, x_dim[0],x_dim[1], 1)) 
    X_test = np.zeros((set_size, x_dim[0],x_dim[1], 1))
    y_test = np.zeros((set_size, classes))
    
    # Raw labels to calculate metrics correctly, when different label is used for training 
    y_raw = np.zeros((set_size*4, classes))
    y_test_raw = np.zeros((set_size, classes))
    
    # Cross validation execution over 5 volts
    for i_5set in range(0, 5):
        Xa[0:set_size, :, :] = X_split[i_5set % 5, :, :, :]
        Xa[set_size:set_size*2, :, :] = X_split[(i_5set+1) % 5, :, :, :]
        Xa[set_size*2:set_size*3, :, :] = X_split[(i_5set+2) % 5, :, :, :]
        Xa[set_size*3:, :, :] = X_split[(i_5set+3) % 5, :, :, :]

        Xa_test = X_split[(i_5set+4) % 5, :, :, :]

        y[0:set_size, :] = y_split[i_5set % 5, :, :]
        y[set_size:set_size*2, :] = y_split[(i_5set+1) % 5, :, :]
        y[set_size*2:set_size*3, :] = y_split[(i_5set+2) % 5, :, :]
        y[set_size*3:, :] = y_split[(i_5set+3) % 5, :, :]

        y_test = y_split[(i_5set+4) % 5, :, :]
        
        y_raw[0:set_size, :] = y_split_raw[i_5set % 5, :, :]
        y_raw[set_size:set_size*2, :] = y_split_raw[(i_5set+1) % 5, :, :]
        y_raw[set_size*2:set_size*3, :] = y_split_raw[(i_5set+2) % 5, :, :]
        y_raw[set_size*3:, :] = y_split_raw[(i_5set+3) % 5, :, :]

        y_test_raw = y_split_raw[(i_5set+4) % 5, :, :]

        # Normalization 1-Zero mean + Unit variance (if nnz_01 is true, than SKIPPED)
        if nnz_01 == False :
            st_dev = np.std(Xa, axis=0)
            st_dev[st_dev == 0] = 1

            mean = np.mean(Xa, axis=0)

            for i_norm1 in range(0, X.shape[0]):
                X[i_norm1, :, :] = np.divide(Xa[i_norm1, :, :] - mean, st_dev)

            for i_norm2 in range(0, X_test.shape[0]):
                X_test[i_norm2, :, :] = np.divide(Xa_test[i_norm2, :, :] - mean, st_dev)
        
        if nnz_01 == True :
            X_test = Xa_test
            X = Xa
        
        # Zero-line evaluation (uniform distribution)
        model_prob = np.array(np.ones(classes) * (1/classes))
        results[metadata:metadata+n_metric] += (np.array(eval_custom(y_raw, model_prob,m_v)) / 5)
        results[metadata+n_metric:metadata+(n_metric*2)]+=(np.array(eval_custom(y_test_raw,model_prob,m_v)) / 5)
        
        # Baseline evaluation (averaging the training set)
        model_prob = np.mean(y_raw, axis=0)
        results[metadata+(n_metric*2):metadata+(n_metric*3)] += (np.array(eval_custom(y_raw,model_prob,m_v))/5)
        results[metadata+(n_metric*3):metadata+baseline] += (np.array(eval_custom(y_test_raw,model_prob,m_v))/5)
        
        # -------------------------Keras execution ---------------------------------

        # Creation of a new model (viz new_model function)
        model = new_model(x_dim, classes, reg_lambda, l_rate, k_mod, obj, margin_value = m_v)

        for i_epoch in range(0, int(n_epoch/av_epoch)):
            
            # Training the model
            mod_t = model.fit(X, y, batch_size=32, nb_epoch=int(av_epoch), verbose=0)
            
            rfi = metadata + baseline + (i_epoch * 2 * n_metric) # current epoch results first index
            
            # Evaluating this model on training data
            results[rfi:rfi+n_metric] += (np.array(eval_custom(y_raw,model.predict_proba(X, verbose=0),m_v))/5)

            # Evaluating this model on validation data
            results[rfi+n_metric:rfi+2*n_metric] += \
                            (np.array(eval_custom(y_test_raw, model.predict_proba(X_test, verbose=0),m_v))/5)
                
        if prin_t :
            print (i_5set+1), " / 5" # Progress print whithin one run
    
    tim_end = timer()
    results[6] = time_el(results[3:6])
    results[7] = tim_end - tim_start
    results[8] = size_keras(model)
    
    return results

In [ ]:
## Settings --------------------------------------------------------

# List of available models and objectives
keras_model_dict = {'linear': 0,     # Just output softmax layer with 6 nodes
                    'dnn_1': 1,
                    'dnn_2': 2,
                    #'dnn_3': 3,
                    #'dnn_4': 4,
                    'cnn_1': 5,
                    'cnn_2': 6,
                    'cnn_3D': 7,
                    #'cnn_4': 8,
                    'cnn_5': 9}
objective_dict = {'kld': 0, '1+2': 1, '1st': 2, 'rank': 3}

# Should be fixed to keep size of the output matrix (at least in terms of one Results file)
number_of_epoch            = 5000     # int......any number
evaluate_each_x_epoch      = 2        # Evaluation will be done after x epochs (must be denominator of n_epoch)

# Hyperparameters to TUNE
random_set                 = 20       # int......any number (should not be changed across different methods)
histogram_dimension_cut    = 100      # Options: 100 95 90 80 70 60 50 40 30 20 (100 -> no cut -> 6615 pixels)
making_histogram_values_01 = False    # If set to true, all histogram non-zero values are set to 1, 0 kept as 0) 
learning_rate              = 0.001    # float....learning_rate > 0 (Tested best value: 0.001)
regularization_lambda      = 0.001      # float....reg_lambda > 0 (for output softmax layer)

# Keras model Settings
keras_model                = 'cnn_3D' # Model selection: 'linear'   'dnn'   'cnn'
kms = []   # Empty list               # Each settings entry (settings for one type of model) must have size 20!
objective                  = 'kld'   # Model will either fit KLD, or classify 1st dominant, or rank 1,2nd
use_raw_input              = True    # Make true, if you want to use raw input data instead of histogram
use_3_classes              = False    # Reduce output label to 3 classes
margin_value               = 0     # Margin value to custom loss function (value 0 ~ 1) (ignore if other loss)

## Execution -------------------------------------------------------

# Getting results for one specified configuration                
results = one_run(n_epoch = number_of_epoch,
                  av_epoch = evaluate_each_x_epoch,
                  
                  random_set = random_set,             
                  cut = histogram_dimension_cut,
                  nnz_01 = making_histogram_values_01,
                  l_rate = learning_rate,
                  reg_lambda = regularization_lambda,
                  
                  k_mod = keras_model_dict[keras_model],
                  obj = objective_dict[objective],
                  inp = (use_raw_input * 1) + (use_3_classes * 2),
                  m_v = margin_value,
                  
                  prin_t = True
                  )

# Save results to .dat file
cur_id = save_result_line('Results_U2.dat', results) # It will append the file (with current ID number)

# Done statement
print 'All DONE!  ->  Results saved with date: ', results[1], '-', results[2], '-', results[3], '  ID: ', cur_id

In [ ]:
## Using preset combinations from external file

# Loading settings file
f = open('prm_config.dat')
set_list = np.genfromtxt(f, delimiter='\t')
f.close

# Execution -------------------------------------------------------
for i_combination in range(0, set_list.shape[0]):

    sett = set_list[i_combination, :]

    results = one_run(n_epoch = sett[0],
                      av_epoch = sett[1],

                      random_set = sett[2],             
                      cut = sett[3],
                      nnz_01 = (1 == sett[4]),
                      l_rate = sett[5],
                      reg_lambda = sett[6],

                      k_mod = sett[7],
                      obj = sett[8],
                      inp = sett[9],
                      m_v = sett[10],
                      
                      prin_t = False
                      )

    # Save results to .dat file
    cur_id = save_result_line('Results_U2.dat', results) # It will append the file (with current ID number)
    
    # Partial done statement
    print 'All DONE!  ->  Results saved with date: ', results[1],'-',results[2],'-',results[3], '  ID: ', cur_id

# Done statement
print 'All DONE!'